## Combination of Correlated measurement

This is a simple example showing the combination of two measurements which are coreelated

These below are the true value and the assumed measurement errors and correlation.

We assume the two measurements are 

$$ x_1 = 8 \pm 2 $$
$$ x_2 = 20 \pm 4 $$

The notebook provides also the capability to generate the two measured value randomly. We start using the given values. 

In [ ]:
double xtrue = 10; 
double sigma1 = 2; 
double sigma2 = 4; 
double rho = 0.98; 
double x1 = 8; 
double x2 = 20; 

##### Random generation of the two measurements
We can generate randomly two correlated measurements with the given true value and given error and correlation.
We generate the values using the multivariate normal distribution for 2 dimension (bi-gaussian).
We use the ROOT *bigaus* function. See the function printout for the parameter definition

In [ ]:
auto f = new TF2("f2","bigaus",0,10,-10,30);
double xr1,xr2;
f->Print();

In [ ]:
f->SetParameters(1,xtrue,sigma1,xtrue,sigma2,rho);

In [ ]:
gRandom->SetSeed(0); // to get always a different value
f->GetRandom2(xr1,xr2);
// uncomment to use the random values or comment to use the fixed values
//x1 = xr1; 
//x2 = xr2; 

In [ ]:
std::cout << " x1 = " << x1 << " +/- " << sigma1 << std::endl;
std::cout << " x2 = " << x2 << " +/- " << sigma2 << std::endl;

#### Combination

We perform now the combination of the two correlated measurement.
The formula to combine them using BLUE is 

$$x_{comb} = w_1 * x_1 + w_2 * x_2$$

where the weights are:

$$w_i = \frac{\sum_j C_{i,j}^{-1} }{\sum_{i,j} C_{i,j}^{-1} }$$

and $C_{i,j}$ is the covariance matrix.

The combined error will be

$$\sigma_{comb} = \frac{1}{\sqrt {\sum_{i,j} C_{i,j}^{-1} }  }$$

We start building the covariance matrix of the two measurements

In [ ]:
TMatrixDSym covMatrix(2);
covMatrix(0,0) = sigma1*sigma1;
covMatrix(1,1) = sigma2*sigma2; 
covMatrix(0,1) = rho * sigma1 * sigma2; 
covMatrix(1,0) = covMatrix(0,1);
covMatrix.Print();

We invert now the covariance matrix to get the weights for the combination. 
The un-normalized weight for the $i$-measurement is equal to the sum of the element in the $i$-th row of the inverse covariance matrix. 

In [ ]:
TMatrixDSym invCMatrix = covMatrix.Invert();

In [ ]:
invCMatrix.Print();

In [ ]:
double w1 = invCMatrix(0,0)+invCMatrix(0,1);
double w2 = invCMatrix(1,0)+invCMatrix(1,1);
double wtot = w1+w2;

Once we have the weights we can get the combined value and error

In [ ]:
double xcomb = (w1 * x1 + w2 * x2)/wtot; 

In [ ]:
double xerror = sqrt( 1./ wtot); 

In [ ]:
std::cout << "combined measurement is " << xcomb << " +/- " << xerror << std::endl; 
std::cout << "combination weights are " << w1/wtot << " , " << w2/wtot << std::endl;

You see that by obtaining a negative weight the combined measurement is outside the two measured values. 

By using the formula for the inverse of the covariance matrix in case of two measurement, we can show that 

$$w_1 = \sigma_{2}^2 - \rho * \sigma_1 * \sigma_2 $$
$$w_2 = \sigma_{1}^2 - \rho * \sigma_1 * \sigma_2 $$

therefore one has a negative weight when $\rho > \sigma_1/\sigma_2$, if $\sigma_2$ is greater than $\sigma_1$, otherwise when $\rho > \sigma_2/\sigma_1$.  